# Project: ASSESSING THE SPREAD AND IMPACT OF COVID-19
## Owner: WHO
### Task: Load Cleaned covid data to database
#### Date: 2021-10-21

Our objective in this notebook is to prepare a database and tables to store covid-19 data. We shall then push the cleaned data to the database.

In [1]:
import pandas as pd
import numpy as np
import mysql.connector as mysql
from mysql.connector import errorcode
from __future__ import print_function

In [2]:
# create database name

DB_NAME = 'WHO_covid_19'

In [3]:
# Create tables

covid_data_table = '''
                    CREATE TABLE covid_19 (
                    entry_id INT NOT NULL,
                    iso_code VARCHAR(10) NOT NULL,
                    continent VARCHAR(30) NOT NULL,
                    location VARCHAR(40) NOT NULL,
                    date_ DATE NOT NULL,
                    total_cases INT NOT NULL, 
                    new_cases INT NOT NULL, 
                    total_deaths INT NOT NULL,
                    new_deaths INT NOT NULL,
                    population INT NOT NULL,
                    median_age FLOAT NOT NULL, 
                    aged_65_older FLOAT NOT NULL,
                    aged_70_older FLOAT NOT NULL,
                    gdp_per_capita FLOAT NOT NULL, 
                    life_expectancy FLOAT NOT NULL,
                    human_development_index FLOAT NOT NULL,
                    PRIMARY KEY (entry_id)
                    )ENGINE=InnoDB
                    
                    '''

In [4]:
# establish connection with our server

config = {
    'user': 'korir',
    'password': 'Wayne1966!',
    'host': 'localhost',
    'raise_on_warnings': True
}

conn = mysql.connect(**config)

In [5]:
# creating cursor object to allow interaction with server

cursor = conn.cursor()

In [6]:
# function for creating databases

def create_database(cursor_object, db_name):
    try:
        cursor.execute("CREATE DATABASE {} CHARACTER SET 'UTF8MB4'".format(db_name))
        print("Database {} created successfully.".format(db_name))
    except mysql.Error as err:
        print("Failed creating database: {}".format(db_name))
        print(err.msg)
        exit(1)


In [7]:
# checking is database already exists and creating one if it doesn't

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.Error as err:
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database {} does not exist".format(DB_NAME))
        create_database(cursor, DB_NAME)
        conn.database = DB_NAME
    else:
        print(err.msg)
        exit(1)

Database WHO_covid_19 does not exist
Database WHO_covid_19 created successfully.


In [8]:
# creating database table and populating it with our data

try:
    cursor.execute(covid_data_table)
    print("Table created successfully")
except mysql.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("Table already exists")
    else:
        print(err.msg)
else:
    print("OK")

Table created successfully
OK


In [9]:
# Query to load data to database

loading_query = '''
                    LOAD DATA INFILE "C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/covid_data.csv"
                    INTO TABLE covid_19
                    COLUMNS TERMINATED BY ','
                    OPTIONALLY ENCLOSED BY '"'
                    ESCAPED BY '"'
                    LINES TERMINATED BY '\r\n'
                    IGNORE 1 LINES;
                    
                   '''

In [10]:
# pushing data to database

try:
    cursor.execute(loading_query)
    print("Success")
except mysql.Error as err:
    print(err.msg)

Success


In [13]:
# committing changes to database 
conn.commit()

In [14]:
# releasing database resources
conn.close()
cursor.close()

True